<a href="https://colab.research.google.com/github/Ninyago53/GPT-2-Chess-Game-Generator/blob/main/gpt_2_Chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Ninyago53/chess-game-generator.git

Cloning into 'chess-game-generator'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 33 (delta 11), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (33/33), 60.99 KiB | 1.74 MiB/s, done.


In [ ]:
!pip install transformers
!pip install chess

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 KB 2.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import json
import os
from transformers import GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, GPT2LMHeadModel

df = pd.read_parquet('/content/chess-game-generator/positions_and_moves.parquet')

moves = df['move'].values.tolist()
positions = df['position'].values.tolist()

text = [val for pair in zip(positions, moves) for val in pair]
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

max_length = tokenizer.model_max_length
text_sequences = [text[i:i+max_length] for i in range(0, len(text), max_length)]

with open('chess_game_text.txt', 'w') as f:
    for seq in text_sequences:
        f.write(" ".join(seq) + "\n")

dataset = TextDataset(
  tokenizer=tokenizer,
  file_path='chess_game_text.txt',
  block_size=128,
  overwrite_cache=True
)

model = GPT2LMHeadModel.from_pretrained('gpt2')

data_collator = DataCollatorForLanguageModeling(
  tokenizer=tokenizer, mlm=False
)

training_args = TrainingArguments(
  output_dir='/content/training_output',
  num_train_epochs=2,
  per_device_train_batch_size=4,
  learning_rate=5e-5,
  logging_dir='/content/logs',
  logging_steps=500
)

trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=dataset,
  data_collator=data_collator
)

trainer.train()

model.save_pretrained('./my_trained_model')


/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 53
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 28
  Number of trainable parameters = 124439808


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in ./my_trained_model/config.json
Configuration saved in ./my_trained_model/generation_config.json
Model weights saved in ./my_trained_model/pytorch_model.bin


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained('./my_trained_model')

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
input_text = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

output_ids = model.generate(input_ids, max_length=100)

output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text)


loading configuration file ./my_trained_model/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "use_cache": true,
  "vocab_size": 50257
}

loading we

rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1 b4/1p2/1p2/1p2/1p2/1p2/1p2/1p2/1p2/1p2/1p2/1p2/1p2/1p2/1p2/1p
